# Morphology

## 1. Disguise proper names in an email

In [35]:
email = """
Уважаемая Эльвира Геннадьевна,

Как шеф-повар, я должен выразить свою обеспокоенность слухами о появлении плесени вблизи некоторых кухонь в корпусе 13 по адресу: Льва Толстого, 42. Обращаю ваше внимание на то, что любые продукты, обрабатываемые, приготавливаемые или потребляемые вблизи плесени, могут быть заражены и небезопасны для употребления.

Если эти сообщения найдут подтверждения, мне придется закрыть несколько столовых. Возможно распределение нагрузки на кухни в корпусах:
№9 площадь Гагарина, 99
№10 улица Южная, 10

С уважением,
Алексей Мартынов.
"""

In [36]:
! pip install -q pymystem3

In [34]:
from pymystem3 import Mystem

mystem = Mystem()

lines = email.split('\n')

def is_proper_name(word):
    parsed_word = mystem.analyze(word)[0]
    if 'analysis' in parsed_word and len(parsed_word['analysis']) > 0 and 'gr' in parsed_word['analysis'][0]:
        gr = parsed_word['analysis'][0]['gr']
        if 'имя' in gr or 'фам' in gr or 'отч' in gr or 'гео' in gr:
            return True
    return False

last_word_was_proper_name = False

for i in range(len(lines)):
    words = lines[i].split()
    for j in range(len(words)):
        if is_proper_name(words[j]) or 'улица' in words[j - 1]:
          if last_word_was_proper_name == False:
            words[j] = '[ИМЯ]'
          else:
            words[j] = ''
          last_word_was_proper_name = True
        else:
          last_word_was_proper_name = False
    lines[i] = ' '.join(words)

masked_email = '\n'.join(lines)

print(masked_email)


Уважаемая [ИМЯ] 

Как шеф-повар, я должен выразить свою обеспокоенность слухами о появлении плесени вблизи некоторых кухонь в корпусе 13 по адресу: [ИМЯ]  42. Обращаю ваше внимание на то, что любые продукты, обрабатываемые, приготавливаемые или потребляемые вблизи плесени, могут быть заражены и небезопасны для употребления.

Если эти сообщения найдут подтверждения, мне придется закрыть несколько столовых. Возможно распределение нагрузки на кухни в корпусах:
№9 площадь [ИМЯ] 99
№10 улица [ИМЯ] 10

С уважением,
[ИМЯ] 



## 2. Replace pymorphy selected object in the wikipedia article with another object (with a declination)

In [38]:
# Example of input data
orig_wikitext = """Стрекозы (лат. Odonáta) — отряд древних летающих насекомых, насчитывающий в мировой фауне свыше 6650 видов. Это относительно крупные насекомые, с подвижной головой, большими глазами, короткими щетинковидными усиками, удлинённым стройным брюшком и четырьмя прозрачными крыльями с густой сетью жилок. Стрекозы — активные специализированные хищники, которые питаются насекомыми, пойманными на лету. Представители отряда широко распространены по миру, встречаясь на всех материках, исключая Антарктиду.
Все представители отряда ведут амфибионтный образ жизни — яйца и личинки развиваются в водной среде, а имаго (взрослые) обитают на суше, освоив воздушную среду и став прекрасными летунами. Взрослые стрекозы не ограничиваются в выборе места обитания одними только берегами водоёмов и могут улетать от них на значительные расстояния, встречаясь на лугах, опушках лесов и даже в населённых пунктах. Развитие с неполным превращением: имеются стадии яйца, личинки и имаго. Личинки (их называют также нимфами или наядами) развиваются в водоёмах различных типов — главным образом в стоячих озёрах, прудах и старицах рек, а также в ручьях, реках и болотистых водоёмах, вплоть до созданных человеком прудов и канав с водой. Как и взрослые стрекозы, личинки являются хищниками. Они охотятся на водных насекомых и прочих беспозвоночных.
Стрекозы имеют большое значение для человека. Велика их роль в регуляции численности кровососущих насекомых, ряда насекомых-вредителей сельского и лесного хозяйства. В некоторых случаях личинки стрекоз могут приносить вред, например, уничтожая мальков в рыбоводных прудах либо составляя им пищевую конкуренцию. Кроме того, личинки некоторых видов могут являться промежуточными хозяевами гельминтов."""

# Example of output data
result_wikitext = """Годзиллы (лат. Odonáta) — отряд древних летающих насекомых, насчитывающий в мировой фауне свыше 6650 видов. Это относительно крупные насекомые, с подвижной головой, большими глазами, короткими щетинковидными усиками, удлинённым стройным брюшком и четырьмя прозрачными крыльями с густой сетью жилок. Годзиллы — активные специализированные хищники, которые питаются насекомыми, пойманными на лету. Представители отряда широко распространены по миру, встречаясь на всех материках, исключая антарктиду. 
Все представители отряда ведут амфибионтный образ жизни — яйца и личинки развиваются в водной среде, а имаго (взрослые) обитают на суше, освоив воздушную среду и став прекрасными летунами. Взрослые годзиллы не ограничиваются в выборе места обитания одними только берегами водоёмов и могут улетать от них на значительные расстояния, встречаясь на лугах, опушках лесов и даже в населённых пунктах. Развитие с неполным превращением: имеются стадии яйца, личинки и имаго. Личинки (их называют также нимфами или наядами) развиваются в водоёмах различных типов — главным образом в стоячих озёрах, прудах и старицах рек, а также в ручьях, реках и болотистых водоёмах, вплоть до созданных человеком прудов и канав с водой. Как и взрослые годзиллы, личинки являются хищниками. Они охотятся на водных насекомых и прочих беспозвоночных. 
Годзиллы имеют большое значение для человека. Велика их роль в регуляции численности кровососущих насекомых, ряда насекомых- вредителей сельского и лесного хозяйства. В некоторых случаях личинки годзилл могут приносить вред, например, уничтожая мальков в рыбоводных прудах либо составляя им пищевую конкуренцию. Кроме того, личинки некоторых видов могут являться промежуточными хозяевами гельминтов."""

In [92]:
import pymorphy2
import re

original_word = "стрекоза"
replacement_word = "годзилла"

morph = pymorphy2.MorphAnalyzer()

def get_forms(word):
  word_lexemes = morph.parse(word)[0].lexeme
  word_lexemes_words = [word_lexeme.word for word_lexeme in word_lexemes]
  word_lexemes_words += [word.title() for word in word_lexemes_words]

  return word_lexemes_words

orig_forms = get_forms(original_word)
repl_forms = get_forms(replacement_word)

result_wikitext = orig_wikitext
for index, word_form in enumerate(orig_forms):
  result_wikitext = re.sub(r'\b' + word_form + r'\b', repl_forms[index], orig_wikitext)

print(result_wikitext)

Стрекозы (лат. Odonáta) — отряд древних летающих насекомых, насчитывающий в мировой фауне свыше 6650 видов. Это относительно крупные насекомые, с подвижной головой, большими глазами, короткими щетинковидными усиками, удлинённым стройным брюшком и четырьмя прозрачными крыльями с густой сетью жилок. Стрекозы — активные специализированные хищники, которые питаются насекомыми, пойманными на лету. Представители отряда широко распространены по миру, встречаясь на всех материках, исключая Антарктиду.
Все представители отряда ведут амфибионтный образ жизни — яйца и личинки развиваются в водной среде, а имаго (взрослые) обитают на суше, освоив воздушную среду и став прекрасными летунами. Взрослые стрекозы не ограничиваются в выборе места обитания одними только берегами водоёмов и могут улетать от них на значительные расстояния, встречаясь на лугах, опушках лесов и даже в населённых пунктах. Развитие с неполным превращением: имеются стадии яйца, личинки и имаго. Личинки (их называют также нимфам

## 3. Comparison of morphological analyzers

In [94]:
! pip install -q natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [136]:
import pymorphy2
import natasha

# text = "Дома дружнее всего, друзьям дружнее всегда."
text = """Стрекозы (лат. Odonáta) — отряд древних летающих насекомых, насчитывающий в мировой фауне свыше 6650 видов. Это относительно крупные насекомые, с подвижной головой, большими глазами, короткими щетинковидными усиками, удлинённым стройным брюшком и четырьмя прозрачными крыльями с густой сетью жилок. Стрекозы — активные специализированные хищники, которые питаются насекомыми, пойманными на лету. Представители отряда широко распространены по миру, встречаясь на всех материках, исключая Антарктиду.
Все представители отряда ведут амфибионтный образ жизни — яйца и личинки развиваются в водной среде, а имаго (взрослые) обитают на суше, освоив воздушную среду и став прекрасными летунами. Взрослые стрекозы не ограничиваются в выборе места обитания одними только берегами водоёмов и могут улетать от них на значительные расстояния, встречаясь на лугах, опушках лесов и даже в населённых пунктах. Развитие с неполным превращением: имеются стадии яйца, личинки и имаго. Личинки (их называют также нимфами или наядами) развиваются в водоёмах различных типов — главным образом в стоячих озёрах, прудах и старицах рек, а также в ручьях, реках и болотистых водоёмах, вплоть до созданных человеком прудов и канав с водой. Как и взрослые стрекозы, личинки являются хищниками. Они охотятся на водных насекомых и прочих беспозвоночных.
Стрекозы имеют большое значение для человека. Велика их роль в регуляции численности кровососущих насекомых, ряда насекомых-вредителей сельского и лесного хозяйства. В некоторых случаях личинки стрекоз могут приносить вред, например, уничтожая мальков в рыбоводных прудах либо составляя им пищевую конкуренцию. Кроме того, личинки некоторых видов могут являться промежуточными хозяевами гельминтов."""

morph1 = pymorphy2.MorphAnalyzer()
morph2 = natasha.MorphVocab()

def process_text(morph_analyzer):
  lemmas = []
  tags = []
  for token in text.split():
    parse_result = morph_analyzer.parse(token)[0]
    # normal_form property will be the same in both analyzers
    if 'MorphVocab' == type(morph_analyzer).__name__:
      lemmas.append(parse_result.normal)
    else:
      lemmas.append(parse_result.word)
    tags.append(parse_result.tag)

  return lemmas, tags

lemmas1, tags1 = process_text(morph1)
lemmas2, tags2 = process_text(morph2)

j = 0
for i in range(len(text.split())):
  if lemmas1[i] != lemmas2[i]:
      print(f"Токен: {text.split()[i]}")
      print(f"Лемма (pymorphy2): {lemmas1[i]}")
      print(f"Лемма (natasha): {lemmas2[i]}")
      print(f"Теги (pymorphy2): {tags1[i]}")
      print(f"Теги (natasha): {tags2[i]}")
      print("-" * 80)
      j += 1
  if j > 2:
    break

Токен: Стрекозы
Лемма (pymorphy2): стрекозы
Лемма (natasha): стрекоза
Теги (pymorphy2): NOUN,anim,femn sing,gent
Теги (natasha): NOUN,anim,femn sing,gent
--------------------------------------------------------------------------------
Токен: древних
Лемма (pymorphy2): древних
Лемма (natasha): древний
Теги (pymorphy2): ADJF,Qual plur,gent
Теги (natasha): ADJF,Qual plur,gent
--------------------------------------------------------------------------------
Токен: летающих
Лемма (pymorphy2): летающих
Лемма (natasha): летать
Теги (pymorphy2): PRTF,impf,intr,pres,actv plur,gent
Теги (natasha): PRTF,impf,intr,pres,actv plur,gent
--------------------------------------------------------------------------------


In [ ]:
!wget -q https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz && tar -xvf ru.tar.gz

ru_small.bin


In [ ]:
import jamspell

corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('ru_small.bin')

True

In [ ]:
# Show jamspell work
correct_examples(corrector.FixFragment, 'JamSpell', 1, True)
# Show jamspell time efficiency
correct_examples(corrector.FixFragment, 'JamSpell')

Corrected:  вот в инете отказал такую интересную статейку предлагаю вашему вниманию
Corrected:  может и в правду лучше тебе молчать чем пытаться сказать
Corrected:  утром мы сидели как сычи а потом каааак начали работать
Elapsed time (seconds) for JamSpell library work (1 loops): 0.018433570861816406
Elapsed time (seconds) for JamSpell library work (300 loops): 2.9468600749969482


#### DeepPavlov

In [ ]:
! pip install -q deeppavlov
! pip install -q sacremoses
! pip install -q https://github.com/kpu/kenlm/archive/master.zip

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from deeppavlov import configs, build_model

In [ ]:
# Download the model of typo fixing
model = build_model(configs.spelling_correction.levenshtein_corrector_ru, download=True)

2023-04-13 23:11:54.574 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz download because of matching hashes
2023-04-13 23:11:55.429 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz download because of matching hashes


In [ ]:
# Show DeepPavlov work
correct_examples(model, 'DeepPavlov', 1, True)
# Show DeepPavlov time efficiency
correct_examples(model, 'DeepPavlov')

Corrected:
вот в инее отказал такую инверсную статейку предлагаю вашему вниманию
может и в правду лучше тебе молчать чем пытаться сказать
утром мы сидели как сычи а потом каааак начали работать
Elapsed time (seconds) for DeepPavlov library work (1 loops): 0.19684505462646484
Elapsed time (seconds) for DeepPavlov library work (300 loops): 35.72472333908081


#### YandexSpeller

In [ ]:
! pip install -q pyaspeller

In [ ]:
from pyaspeller import YandexSpeller
speller = YandexSpeller()

In [ ]:
# Show YandexSpeller work
correct_examples(speller.spelled, 'YandexSpeller', 1, True)
# Show YandexSpeller time efficiency
correct_examples(model, 'YandexSpeller')

Corrected:  вот в инете откопал такую интересную статейку предлагаю вашему вниманию
Corrected:  может и в правду лучше тебе молчать чем пытаться сказать
Corrected:  утром мы сидели как сычи а потом каааак начали работать
Elapsed time (seconds) for YandexSpeller library work (1 loops): 1.7286052703857422
Elapsed time (seconds) for YandexSpeller library work (300 loops): 80.13619661331177


## 2. Damerau-Levenshtein function

In [ ]:
! pip install -q python-Levenshtein

In [ ]:
import Levenshtein

fixed_examples = [
    "вот в инете откопал такую интересную статейку предлагаю вашему вниманию",
    "может и в правду лучше тебе молчать чем пытаться сказать",
    "утром мы сидели как сычи а потом как начали работать",
]

start_time = time.time()
for i in range(1000):
  for j, example in enumerate(examples):
    distance = Levenshtein.distance(examples[j], fixed_examples[j])

end_time = time.time() # Record the end time
elapsed_time = end_time - start_time # Calculate elapsed time
print("Elapsed time (seconds) for Levenshtein.distance: {}".format(elapsed_time))

Elapsed time (seconds) for Levenshtein.distance: 0.005997419357299805


## 3. Typos due to key slips on the keyboard function

In [ ]:
import random

def generate_typo(word):
    keyboard = {
        'q': ['w', 'a'],
        'w': ['q', 'e', 'a', 's'],
        'e': ['w', 'r', 's', 'd'],
        'r': ['e', 't', 'd', 'f'],
        't': ['r', 'y', 'f', 'g'],
        'y': ['t', 'u', 'g', 'h'],
        'u': ['y', 'i', 'h', 'j'],
        'i': ['u', 'o', 'j', 'k'],
        'o': ['i', 'p', 'k', 'l'],
        'p': ['o', 'l'],
        'a': ['q', 'w', 's', 'z'],
        's': ['w', 'a', 'd', 'z', 'x'],
        'd': ['e', 's', 'f', 'x', 'c'],
        'f': ['r', 'd', 'g', 'c', 'v'],
        'g': ['t', 'f', 'h', 'v', 'b'],
        'h': ['y', 'g', 'j', 'b', 'n'],
        'j': ['u', 'h', 'k', 'n', 'm'],
        'k': ['i', 'j', 'l', 'm'],
        'l': ['o', 'k', 'p'],
        'z': ['a', 's', 'x'],
        'x': ['s', 'd', 'z', 'c'],
        'c': ['d', 'f', 'x', 'v'],
        'v': ['f', 'g', 'c', 'b'],
        'b': ['g', 'h', 'v', 'n'],
        'n': ['h', 'j', 'b', 'm'],
        'm': ['j', 'k', 'n']
    }

    word_list = list(word)
    
    # Pick random place of type among the letters
    idx = random.randint(0, len(word) - 1)
    
    # Get a char from this position
    char = word_list[idx]
    
    # We generate a typo by randomly replacing a character with a neighboring one on the keyboard
    if char.lower() in keyboard:
        typo_char = random.choice(keyboard[char.lower()])
        word_list[idx] = typo_char
    
    return ''.join(word_list)

word = "hello"
typo_word = generate_typo(word)
print(typo_word)

hwllo


## 4. Comparing vectors composed of frequencies of letters in a word (CountVectorizer)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compare_word_vectors(word1, word2):
    # Create CountVectorizer object
    vectorizer = CountVectorizer(analyzer='char')
    
    # Combine the two words into a list
    word_list = [word1, word2]
    
    # Fit and transform the word list to obtain word vectors
    word_vectors = vectorizer.fit_transform(word_list)
    
    # Compute cosine similarity between the two word vectors
    similarity = cosine_similarity(word_vectors[0], word_vectors[1])
    
    # Return cosine similarity score
    return similarity[0][0]

In [ ]:
start_time = time.time()
for i in range(1000):
  for j, example in enumerate(examples):
    similarity_score = compare_word_vectors(examples[j], fixed_examples[j])

end_time = time.time() # Record the end time
elapsed_time = end_time - start_time # Calculate elapsed time
print("Elapsed time (seconds) for compare_word_vectors: {}".format(elapsed_time))

print("Cosine similarity between '{}' and '{}': {}".format(examples[j], fixed_examples[j], similarity_score))

Elapsed time (seconds) for compare_word_vectors: 8.592718362808228
Cosine similarity between 'утром мы сидели как сычи а потом каааак начали работать' and 'утром мы сидели как сычи а потом как начали работать': 0.987317065004051
